In [1]:
import sys 
import logging
sys.path.append('/Users/jerald/Documents/Dir/Python/Stocks')
from bin.main import get_path 
from models.bsm.bsModel import bs_df
from bin.plots.utils import pretty_print

import pandas as pd 
import numpy as np 
import yfinance as yf 
import datetime as dt 
from tqdm import tqdm
import sqlite3 as sql
from IPython.display import display
from typing import Any, Dict, List, Tuple, Optional


connections = get_path('../../../')


In [2]:
def get_query_str(stock: str) -> str:
    q = f'''
        SELECT 
        datetime(gatherdate) AS gatherdate,
        CAST(sum(volume) AS INT) AS total_vol,
        CAST(SUM(CASE WHEN type = 'Call' THEN volume ELSE 0 END) AS INT) AS call_vol,
        CAST(SUM(CASE WHEN type = 'Put' THEN volume ELSE 0 END) AS INT) AS put_vol,
        CAST(sum(openinterest) AS INT) AS total_oi,
        CAST(SUM(CASE WHEN type = 'Call' THEN openinterest ELSE 0 END) AS INT) AS call_oi, 
        CAST(SUM(CASE WHEN type = 'Put' THEN openinterest ELSE 0 END) AS INT) AS put_oi
        FROM {stock}
        GROUP BY datetime(gatherdate)
        ORDER BY gatherdate ASC
        '''
    return q


def sql_connections(query:str, connections: Dict[str, str] = connections, db: str = 'option_db') -> pd.DataFrame:
    conn = sql.connect(connections[db])
    cursor = conn.cursor()
    cursor.execute(query)
    df = pd.DataFrame(cursor.fetchall(), columns=[col[0] for col in cursor.description])
    if 'gatherdate' in df.columns:
        df['gatherdate'] = pd.to_datetime(df['gatherdate'])
    conn.close()
    return df

def cp_df(stock:str) -> pd.DataFrame:
    query = get_query_str(stock)
    return sql_connections(query)

def get_stock(stock:str, db:str = 'option_db') -> pd.DataFrame:
    query = f'select * from {stock}'
    df = sql_connections(query, connections, db)
    if 'gatherdate' in df.columns:
        df['gatherdate'] = pd.to_datetime(df['gatherdate'])
    return df 

def change_variables(df:pd.DataFrame) -> pd.DataFrame:
    pass

In [4]:
df = cp_df('spy')

In [5]:
# Get the last data entry for each day 
df['date'] = df['gatherdate'].dt.date
df = df.set_index('date')
gbdf = df.groupby(df.index).last().reset_index(drop=True).set_index('gatherdate').diff()
gbdf.columns = [ f'{col}_chng' for col in gbdf.columns]

final_change = df.merge(gbdf, left_on = 'gatherdate', right_index=True).reset_index(drop=True).tail()

In [6]:
pretty_print(final_change[['gatherdate', 'total_vol', 'total_vol_chng', 'total_oi', 'total_oi_chng']])

,gatherdate,total_vol,total_vol_chng,total_oi,total_oi_chng
50,2025-04-11 14:13:29,"6,723,612.00","-3,544,406.00","20,017,414.00","554,210.00"
51,2025-04-14 14:58:59,"7,728,914.00","1,005,302.00","19,537,379.00","-480,035.00"
52,2025-04-15 14:55:32,"7,338,983.00","-389,931.00","19,599,080.00","61,701.00"
53,2025-04-16 14:07:12,"5,820,277.00","-1,518,706.00","19,574,249.00","-24,831.00"
54,2025-04-17 14:07:35,"6,971,774.00","1,151,497.00","19,691,090.00","116,841.00"


In [7]:
pretty_print(get_stock('spy', 'vol_db')[['gatherdate', 'total_vol', 'total_vol_chng', 'total_oi', 'total_oi_chng']].tail())

,gatherdate,total_vol,total_vol_chng,total_oi,total_oi_chng
720,2025-04-11 14:13:29,"6,723,612.00","-3,544,406.00","20,017,414.00","554,210.00"
721,2025-04-14 14:58:59,"7,728,914.00","1,005,302.00","19,537,379.00","-480,035.00"
722,2025-04-15 14:55:32,"7,338,983.00","-389,931.00","19,599,080.00","61,701.00"
723,2025-04-16 14:07:12,"5,820,277.00","-1,518,706.00","19,574,249.00","-24,831.00"
724,2025-04-17 14:07:35,"6,971,774.00","1,151,497.00","19,691,090.00","116,841.00"


In [8]:
pretty_print(get_stock('spy where dte_flag = "total"', 'vol2_db').tail()[['gatherdate', 'total_vol', 'total_vol_chng', 'total_oi', 'total_oi_chng']])

,gatherdate,total_vol,total_vol_chng,total_oi,total_oi_chng
50,2025-04-11 14:13:29,"6,723,612.00","-3,544,406.00","20,017,414.00","554,210.00"
51,2025-04-14 14:58:59,"7,728,914.00","1,005,302.00","19,537,379.00","-480,035.00"
52,2025-04-15 14:55:32,"7,338,983.00","-389,931.00","19,599,080.00","61,701.00"
53,2025-04-16 14:07:12,"5,820,277.00","-1,518,706.00","19,574,249.00","-24,831.00"
54,2025-04-17 14:07:35,"6,971,774.00","1,151,497.00","19,691,090.00","116,841.00"


In [16]:
pretty_print(get_stock('qqq where date(gatherdate) > "2023-01-01"', 'vol2_db').head(8)[['gatherdate','dte_flag','total_vol', 'total_vol_chng', 'total_oi', 'total_oi_chng', 'call_oi_chng', 'put_oi_chng']])

,gatherdate,dte_flag,total_vol,total_vol_chng,total_oi,total_oi_chng,call_oi_chng,put_oi_chng
0,2023-01-03 14:33:28,LTE,"21,191.00","-4,754.00","77,166.00","11,020.00",278.00,"10,742.00"
1,2023-01-03 14:33:28,total,"21,191.00","-4,754.00","77,166.00","11,020.00",278.00,"10,742.00"
2,2023-01-04 10:38:54,LTE,"7,145.00","-14,046.00","77,267.00",101.00,107.00,-6.00
3,2023-01-04 10:38:54,total,"7,145.00","-14,046.00","77,267.00",101.00,107.00,-6.00
4,2023-01-05 11:43:48,LTE,"7,534.00",389.00,"77,391.00",124.00,49.00,75.00
5,2023-01-05 11:43:48,total,"7,534.00",389.00,"77,391.00",124.00,49.00,75.00
6,2023-01-06 12:11:00,LTE,"7,026.00",-508.00,"77,670.00",279.00,234.00,45.00
7,2023-01-06 12:11:00,total,"7,026.00",-508.00,"77,670.00",279.00,234.00,45.00
